# Import Library and Module necessary for Model

**Tổng quan mô hình**
--
---
- Mô hình Naive Bayes giả định rằng các đặc trưng trong dữ liệu độc lập với nhau.
- Với bài toán phân loại mô hình Naive Bayes đem lại một hiệu suất tốt với cách triển khai đơn giản.

In [81]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

import os

import tensorflow_hub as hub
# tensor flow module
import tensorflow as tf
import tensorflow_probability as tfp

# matplotlib
from matplotlib import colors
from matplotlib import pyplot as plt

# word vectorizor
# first converts the text into a matrix of word counts
# then transforms these counts by normalizing them based on the term frequency
from sklearn.feature_extraction.text import TfidfVectorizer

from google.colab import drive

drive.mount('/content/drive', force_remount=True)
for dirname, _, filenames in os.walk('/content/drive/My Drive/Colab Notebooks/nlp-getting-started'
):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/nlp-getting-started/sample_submission.csv
/content/drive/My Drive/Colab Notebooks/nlp-getting-started/test.csv
/content/drive/My Drive/Colab Notebooks/nlp-getting-started/train.csv


# Data Preprocessing

## Import Dataset

---
- Bộ dữ liệu phân loại nhị phân được dùng trong mô hình là Natural Language Processing With Disaster Tweets. [Link Drive Dataset ở đây.](https://drive.google.com/drive/folders/19bqWjvVvlYBGucs8zFWv1-xr1FjN_u_7) Hoặc có sẵn file csv trong Folder, thay đổi đường dẫn để lấy Dataset.
- Sau khi Get Data chia Data làm 2 tập train và test.

In [82]:
train_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/nlp-getting-started/train.csv")
test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/nlp-getting-started/test.csv")

## Embeding Tweet Text.

---
- Khi làm việc với dữ liệu là ngôn ngữ tự nhiên, ta phải Embedding các *câu* (sentence), *từ ngữ* (text) thành các *vector*.
- Trong mô hình này ta dùng Tensorflow universal-sentence-encoder để Embedding.

In [83]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/3")
X_train_embeddings = embed(train_df["text"].values)
X_test_embeddings = embed(test_df["text"].values)

## Tensorflow Numpy Matrics
---
- Để có thể huấn luyện mô hình Naive Bayes, ta chuẩn hoá các mẫu dữ liệu thành ma trận với *hàng* (các mẫu độc lập) và *cột* (các đặc trưng) nhằm tối ưu hoá việc tổ chức, xử lý, tính toán. Với bộ dữ liệu lớn hay nhỏ đều có thể dùng được.

In [84]:
X_train_matrix = X_train_embeddings['outputs'].numpy()
X_test_matrix = X_test_embeddings['outputs'].numpy()

Y_train = tf.constant(train_df["target"])

# Build And Train Naive Bayes Classifier

## Build Naive Bayes Classifier
---
- # `fit()`: Huấn luyện học máy mô hình Naive Bayes.
- # `predict()`: Phân loại dữ liệu dựa trên mô hình Naive Bayes.

In [88]:
class TFNaiveBayesClassifier:
    dist = None

    # X is the matrix containing the vectors for each sentence
    # y is the list target values in the same order as the X matrix
    def fit(self, X, y):
        unique_y = np.unique(y) # unique target values: 0,1
        print(unique_y)
        # `points_by_class` is a numpy array the size of
        # the number of unique targets.
        # in each item of the list is another list that contains the vector
        # of each sentence from the same target value
        points_by_class = np.asarray([np.asarray(
            [np.asarray(
                X.iloc[x,:]) for x in range(0,len(y)) if y[x] == c]) for c in unique_y])
        mean_list=[]
        var_list=[]
        for i in range(0, len(points_by_class)):
            mean_var, var_var = tf.nn.moments(tf.constant(points_by_class[i]), axes=[0])
            mean_list.append(mean_var)
            var_list.append(var_var)
        mean=tf.stack(mean_list, 0)
        var=tf.stack(var_list, 0)
        # Create a 3x2 univariate normal distribution with the
        # known mean and variance
        self.dist = tfp.distributions.Normal(loc=mean, scale=tf.sqrt(var))

    def predict(self, X):
        assert self.dist is not None
        nb_classes, nb_features = map(int, self.dist.scale.shape)

        # uniform priors
        priors = np.log(np.array([1. / nb_classes] * nb_classes))

        # Conditional probabilities log P(x|c)
        # (nb_samples, nb_classes, nb_features)
        all_log_probs = self.dist.log_prob(
            tf.reshape(
                tf.tile(X, [1, nb_classes]), [-1, nb_classes, nb_features]))
        # (nb_samples, nb_classes)
        cond_probs = tf.reduce_sum(all_log_probs, axis=2)

        # posterior log probability, log P(c) + log P(x|c)
        joint_likelihood = tf.add(priors, cond_probs)

        # normalize to get (log)-probabilities
        norm_factor = tf.reduce_logsumexp(
            joint_likelihood, axis=1, keepdims=True)
        log_prob = joint_likelihood - norm_factor
        # exp to get the actual probabilities
        return tf.exp(log_prob)

## Initialize Naive Bayes Model

In [89]:
tf_nb = TFNaiveBayesClassifier()

## Train Naive Bayes With Train Data

In [90]:
tf_nb.fit(X=pd.DataFrame(X_train_matrix), y=Y_train)

[0 1]


KeyError: "None of [Index([False, False, False, False, False, False, False, False, False, False,\n       ...\n       False, False, False, False, False, False, False, False, False, False],\n      dtype='bool', length=7613)] are in the [columns]"

## Predict Test Data

In [ ]:
y_pred = tf_nb.predict(X_test_matrix)